## Feature Engineering using Featuretools

#### Import library

In [1]:
import pandas as pd
import featuretools as ft

C:\Users\acer\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [4]:
# Load CSV files and remove duplicates
customers = pd.read_csv("customers.csv").drop_duplicates(subset="Customer ID")
orders = pd.read_csv("orders.csv").drop_duplicates(subset="Order ID")
products = pd.read_csv("products.csv").drop_duplicates(subset="Product ID")
order_details = pd.read_csv("order_details.csv")

# Ensure date columns are properly parsed
orders["Order Date"] = pd.to_datetime(orders["Order Date"], format="%m/%d/%Y", errors="coerce")
orders["Ship Date"] = pd.to_datetime(orders["Ship Date"], format="%m/%d/%Y", errors="coerce")

In [5]:
customer_types = {
    "Customer ID": "Categorical",
    "Customer Name": "Categorical",
    "Country": "Categorical",
    "State": "Categorical",
    "Region": "Categorical",
}

order_types = {
    "Order ID": "Categorical",
    "Order Date": "Datetime",
    "Ship Date": "Datetime",
    "Ship Mode": "Categorical",
}

product_types = {
    "Product ID": "Categorical",
    "Category": "Categorical",
    "Sub-Category": "Categorical",
    "Product Name": "Categorical",
}

order_details_types = {
    "Order ID": "Categorical",
    "Product ID": "Categorical",
    "Customer ID": "Categorical",
    "Quantity": "Integer",
    "Discount": "Double",
    "Profit": "Double",
    "Sales": "Double",
}

In [6]:
es = ft.EntitySet(id="ecommerce")

In [7]:
es = es.add_dataframe(
    dataframe_name="customers",
    dataframe=customers,
    index="Customer ID",  # Primary key
    logical_types=customer_types
)

# Add orders dataframe to EntitySet
es = es.add_dataframe(
    dataframe_name="orders",
    dataframe=orders,
    index="Order ID",  # Primary key
    logical_types=order_types,
    time_index="Order Date"  # Time index
)

# Add products dataframe to EntitySet
es = es.add_dataframe(
    dataframe_name="products",
    dataframe=products,
    index="Product ID",  # Primary key
    logical_types=product_types
)

order_details = order_details.reset_index()  # Create a unique index
order_details.rename(columns={"index": "order_details_id"}, inplace=True)

# Add order_details dataframe to EntitySet
es = es.add_dataframe(
    dataframe_name="order_details",
    dataframe=order_details,
    index="order_details_id",
    logical_types=order_details_types
)

In [8]:
print(es)

Entityset: ecommerce
  DataFrames:
    customers [Rows: 793, Columns: 5]
    orders [Rows: 5009, Columns: 4]
    products [Rows: 1862, Columns: 4]
    order_details [Rows: 9994, Columns: 8]
  Relationships:
    No relationships


In [9]:
# Define relationships using the desired format
es = es.add_relationship("customers", "Customer ID", "order_details", "Customer ID")
es = es.add_relationship("orders", "Order ID", "order_details", "Order ID")
es = es.add_relationship("products", "Product ID", "order_details", "Product ID")

In [10]:
print(es)

Entityset: ecommerce
  DataFrames:
    customers [Rows: 793, Columns: 5]
    orders [Rows: 5009, Columns: 4]
    products [Rows: 1862, Columns: 4]
    order_details [Rows: 9994, Columns: 8]
  Relationships:
    order_details.Customer ID -> customers.Customer ID
    order_details.Order ID -> orders.Order ID
    order_details.Product ID -> products.Product ID


In [11]:
feature_matrix, feature_defs = ft.dfs(
    entityset=es,                              # The EntitySet
    target_dataframe_name="order_details"
)

C:\Users\acer\anaconda3\lib\site-packages\featuretools\computational_backends\feature_set_calculator.py:785: FutureWarning: The provided callable <function min at 0x00000218C8D2B250> is currently using SeriesGroupBy.min. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "min" instead.
  ).agg(to_agg)
C:\Users\acer\anaconda3\lib\site-packages\featuretools\computational_backends\feature_set_calculator.py:785: FutureWarning: The provided callable <function std at 0x00000218C8D2BB50> is currently using SeriesGroupBy.std. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "std" instead.
  ).agg(to_agg)
C:\Users\acer\anaconda3\lib\site-packages\featuretools\computational_backends\feature_set_calculator.py:785: FutureWarning: The provided callable <function max at 0x00000218C8D2B130> is currently using SeriesGroupBy.max. In a future version of pandas, the provid

In [12]:
# Display the resulting feature matrix
print(feature_matrix.head())

                        Order ID       Product ID     Sales  Quantity  \
order_details_id                                                        
0                 CA-2016-152156  FUR-BO-10001798  261.9600         2   
1                 CA-2016-152156  FUR-CH-10000454  731.9400         3   
2                 CA-2016-138688  OFF-LA-10000240   14.6200         2   
3                 US-2015-108966  FUR-TA-10000577  957.5775         5   
4                 US-2015-108966  OFF-ST-10000760   22.3680         2   

                  Discount    Profit Customer ID customers.Customer Name  \
order_details_id                                                           
0                     0.00   41.9136    CG-12520             Claire Gute   
1                     0.00  219.5820    CG-12520             Claire Gute   
2                     0.00    6.8714    DV-13045         Darrin Van Huff   
3                     0.45 -383.0310    SO-20335          Sean O'Donnell   
4                     0.20    2.

In [13]:
print(feature_matrix.columns.tolist())

['Order ID', 'Product ID', 'Sales', 'Quantity', 'Discount', 'Profit', 'Customer ID', 'customers.Customer Name', 'customers.Country', 'customers.State', 'customers.Region', 'orders.Ship Mode', 'products.Category', 'products.Sub-Category', 'products.Product Name', 'customers.COUNT(order_details)', 'customers.MAX(order_details.Discount)', 'customers.MAX(order_details.Profit)', 'customers.MAX(order_details.Quantity)', 'customers.MAX(order_details.Sales)', 'customers.MEAN(order_details.Discount)', 'customers.MEAN(order_details.Profit)', 'customers.MEAN(order_details.Quantity)', 'customers.MEAN(order_details.Sales)', 'customers.MIN(order_details.Discount)', 'customers.MIN(order_details.Profit)', 'customers.MIN(order_details.Quantity)', 'customers.MIN(order_details.Sales)', 'customers.MODE(order_details.Order ID)', 'customers.MODE(order_details.Product ID)', 'customers.NUM_UNIQUE(order_details.Order ID)', 'customers.NUM_UNIQUE(order_details.Product ID)', 'customers.SKEW(order_details.Discount

In [14]:
# Save the resulting feature matrix
feature_matrix.to_csv("engineered_features.csv", index=False)